<a href="https://colab.research.google.com/github/ignadv1986/IP-MS-data-analysis-and-Alphafold-validation/blob/main/CRAMP1_GON4L_full_VS_fragment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Opening the file and checking the column names
import json

with open("cramp1_gon4l_full.json") as f:
    data = json.load(f)

# Show top-level keys
print("Top-level keys in JSON:")
print(data.keys())


Top-level keys in JSON:
dict_keys(['atom_chain_ids', 'atom_plddts', 'contact_probs', 'pae', 'token_chain_ids', 'token_res_ids'])


In [7]:
import json
import numpy as np
import pandas as pd

def get_residue_plddt_stats(json_path):
    with open(json_path) as f:
        data = json.load(f)

    plddts = data["atom_plddts"]
    res_ids = data["token_res_ids"]
    chain_ids = data["token_chain_ids"]

    if len(plddts) != len(res_ids):
        print(f"⚠️ Length mismatch in {json_path}: {len(plddts)} pLDDTs vs {len(res_ids)} residues")
        plddts = plddts[:len(res_ids)]

    # Build (residue_id, chain_id) → pLDDT
    residue_plddt = {}
    for i in range(len(plddts)):
        key = (res_ids[i], chain_ids[i])
        residue_plddt[key] = plddts[i]

    scores = list(residue_plddt.values())
    mean_score = np.mean(scores)
    perc_above_70 = np.sum(np.array(scores) > 70) / len(scores) * 100

    return mean_score, perc_above_70, len(scores)

# Run for both files
full_mean, full_above70, full_total = get_residue_plddt_stats("cramp1_gon4l_full.json")
frag_mean, frag_above70, frag_total = get_residue_plddt_stats("cramp1_gon4l_fragment.json")

# Store in DataFrame
summary_df = pd.DataFrame({
    "Model": ["Full Model", "Fragment Model"],
    "Total Residues": [full_total, frag_total],
    "Mean pLDDT": [full_mean, frag_mean],
    "% Residues >70": [full_above70, frag_above70]
})

# Save to CSV
summary_df.to_csv("plddt_summary_corrected.csv", index=False)


⚠️ Length mismatch in cramp1_gon4l_full.json: 26894 pLDDTs vs 3510 residues
⚠️ Length mismatch in cramp1_gon4l_fragment.json: 8208 pLDDTs vs 1070 residues
            Model  Total Residues  Mean pLDDT  % Residues >70
0      Full Model            3510   56.530883       38.774929
1  Fragment Model            1070   72.311963       63.457944
